# Projet Machine Learning : Prédiction du Succès des Campagnes de Télémarketing Bancaire

**Auteur:** Lyes SID ALI  -  Cédric LES BIENS  -  Nour MAALI  
**Date:** Janvier 2026  
**Dataset:** Bank Marketing Dataset  
**Enseignant:** Dario COLAZZO  

## Plan
1. Description du Problème et Objectifs
2. Import et chargement
3. Exploration et Analyse des Données
4. Preprocessing et Feature Engineering
5. Modélisation
6. Évaluation et Comparaison des Modèles
7. Validation Finale
8. Conclusions et Perspectives

## 1. Description du problème et objectifs
### 1.1 Contexte
Une institution bancaire portugaise a mené des campagnes de marketing direct basées sur des appels téléphoniques pour promouvoir des dépôts à terme. Ces campagnes ont nécessité de multiples contacts avec les mêmes clients pour déterminer s'ils souscrivaient ou non au produit bancaire proposé.

### 1.2 Problème à Résoudre
Prédire si un client va souscrire à un dépôt à terme bancaire (variable **y**) en fonction de ses caractéristiques personnelles, du contexte de la campagne et des indicateurs socio-économiques.

### 1.3 Type de Problème
**Classification Binaire**
- Il s'agit d'un problème de classification car la variable cible est catégorielle binaire (yes/no), et ce n'est pas une regression car nous ne prédisons pas une valeur numérique continue
- **Objectif :** Classifier chaque client en deux catégories : 
  - Classe positive (1) : le client souscrit au dépôt (**y = yes**)
  - Classe négative (0) : le client ne souscrit pas (**y = no**)

### 1.4 Variables et Features

**Variable Cible (la target) :**
- **y** : Le client a-t-il souscrit à un dépôt à terme ? yes/no

**Features (20 variables d'entrée) :**

**A. Données du client bancaire (7 features)**
- `age` : Age
- `job` : Type d'emploi 
- `marital` : Statut matrimonial 
- `education` : Niveau d'éducation 
- `default` : Crédit en défaut  
- `housing` : Prêt immobilier 
- `loan` : Prêt personnel 

**B. Données du dernier contact de la campagne (4 features)**
- `contact` : Type de communication
- `month` : Mois du dernier contact 
- `day_of_week` : Jour de la semaine 
- `duration` : Durée du dernier contact en secondes 

**C. Autres attributs de la campagne (4 features)**
- `campaign` : Nombre de contacts durant cette campagne 
- `pdays` : Jours depuis le dernier contact d'une campagne précédente
- `previous` : Nombre de contacts avant cette campagne 
- `poutcome` : Résultat de la campagne précédente 

**D. Données socio-économiques (5 features)**
- `emp.var.rate` : Taux de variation de l'emploi 
- `cons.price.idx` : Indice des prix à la consommation
- `cons.conf.idx` : Indice de confiance des consommateurs
- `euribor3m` : Taux Euribor à 3 mois 
- `nr.employed` : Nombre d'employés 

### 1.5 Algorithmes à Utiliser

Nous allons implémenter et comparer 5 algorithmes différents :

1. **Régression Logistique:**  Modèle de classification linéaire simple
2. **Decision Tree :**  Algorithme basé sur des règles de décision
3. **Naive Bayes:**  Classification utilisant les probabilités basée sur le théorème de Bayes
4. **Perceptron:**  Algorithme de classification linéaire
5. **K-Nearest Neighbors (KNN):**  Classification basée sur la similarité avec les k voisins les plus proches

### 1.6 Métriques d'Évaluation

Étant donné qu'il s'agit d'un problème de classification binaire potentiellement déséquilibré, nous utiliserons les métriques :

- **Accuracy** : Taux de bonnes prédictions global 
- **Precision** : Proportion de vrais positifs parmi les prédictions positives 
- **Recall** : Proportion de vrais positifs parmi tous les positifs réels  
- **F1-Score** : Moyenne de Precision et Recall 
- **Matrice de Confusion** : Visualisation des vrais/faux positifs/négatifs
- **Courbe ROC** : Pour visualiser le compromis sensibilité/spécificité

### 1.7 Stratégie de Validation (Model Selection)

- **Train-Test Split** : 80% entraînement, 20% test
- **Validation Croisée K-flod** : 5-fold cross-validation pour évaluer la robustesse

## 2. Import et Chargement

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             roc_auc_score, confusion_matrix, roc_curve, auc)
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Imports OK")

In [ ]:
# Chargement du dataset
df = pd.read_csv('bank-additional/bank-additional-full.csv', sep=';')
print(f"Dataset chargé : {df.shape[0]} lignes, {df.shape[1]} colonnes")

## 3. Exploration des Données

In [ ]:
# Apercu des données
print("Apercu des données")

df.head(10)

In [ ]:
# Infos générales
print("Infos dataset :")
print(df.info())
print("\nStats :")
df.describe()

In [ ]:
# Valeurs manquantes
missing = df.isnull().sum()
if missing.sum() == 0:
    print("Pas de valeurs manquantes")
else:
    print(missing[missing > 0])

In [ ]:
# Distribution variable cible
target_counts = df['y'].value_counts()
print("Distribution :")
print(target_counts)
print(f"\nProportions :\n{df['y'].value_counts(normalize=True)*100}")

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].bar(target_counts.index, target_counts.values, color=['#ff6b6b', '#4ecdc4'])
axes[0].set_title('Distribution Cible')
axes[0].set_ylabel('Nombre')
for i, v in enumerate(target_counts.values):
    axes[0].text(i, v + 500, str(v), ha='center')

axes[1].pie(target_counts.values, labels=target_counts.index, autopct='%1.1f%%', 
            colors=['#ff6b6b', '#4ecdc4'])
axes[1].set_title('Proportions')

plt.tight_layout()
plt.show()

ratio = target_counts['no'] / target_counts['yes']
print(f"\nRatio déséquilibre : {ratio:.1f} (no/yes)")
print("Classes très déséquilibrées, faudra en tenir compte")

In [ ]:
# Variables catégorielles
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
cat_cols.remove('y')

print(f"Variables catégorielles : {cat_cols}\n")

# Nombre de catégories
for col in cat_cols:
    print(f"{col}: {df[col].nunique()} catégories")
    
# Valeurs inconnues
print("\nValeurs unkonwn :")
for col in cat_cols:
    unknown = (df[col] == 'unknown').sum()
    if unknown > 0:
        print(f"{col}: {unknown} ({unknown/len(df)*100:.1f}%)")

In [ ]:
# Visualisation variables catégorielles
fig, axes = plt.subplots(3, 3, figsize=(15, 10))
axes = axes.ravel()

cols_plot = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

for idx, col in enumerate(cols_plot):
    counts = df[col].value_counts()
    axes[idx].barh(range(len(counts)), counts.values)
    axes[idx].set_yticks(range(len(counts)))
    axes[idx].set_yticklabels(counts.index)
    axes[idx].set_title(col)
    
plt.tight_layout()
plt.show()

In [ ]:
# variables numeriques
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"Variables numériques : {num_cols}\n")
df[num_cols].describe().T

In [ ]:
# Distribution variables numériques
fig, axes = plt.subplots(4, 3, figsize=(15, 12))
axes = axes.ravel()

for idx, col in enumerate(num_cols):
    axes[idx].hist(df[col], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    axes[idx].set_title(col)
    axes[idx].set_ylabel('Fréquence')
    
for idx in range(len(num_cols), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

In [ ]:
# Correlations
plt.figure(figsize=(11, 8))
corr = df[num_cols].corr()
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlations')
plt.tight_layout()
plt.show()

# Correlations fortes
print("Correlations fortes (>0.7) :")
for i in range(len(corr.columns)):
    for j in range(i+1, len(corr.columns)):
        if abs(corr.iloc[i, j]) > 0.7:
            print(f"{corr.columns[i]} <-> {corr.columns[j]}: {corr.iloc[i, j]:.2f}")

In [ ]:
# Relation features / cible
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Age
axes[0, 0].hist([df[df['y']=='yes']['age'], df[df['y']=='no']['age']], 
                bins=30, label=['yes', 'no'], color=['#4ecdc4', '#ff6b6b'], alpha=0.7)
axes[0, 0].set_title('Age')
axes[0, 0].legend()

# Campaign
axes[0, 1].hist([df[df['y']=='yes']['campaign'], df[df['y']=='no']['campaign']], 
                bins=20, label=['yes', 'no'], color=['#4ecdc4', '#ff6b6b'], alpha=0.7, range=(0, 20))
axes[0, 1].set_title('Contacts campagne')
axes[0, 1].legend()

# Previous
axes[0, 2].hist([df[df['y']=='yes']['previous'], df[df['y']=='no']['previous']], 
                bins=15, label=['yes', 'no'], color=['#4ecdc4', '#ff6b6b'], alpha=0.7, range=(0, 10))
axes[0, 2].set_title('Contacts précédents')
axes[0, 2].legend()

#job
job_target = pd.crosstab(df['job'], df['y'], normalize='index') * 100
job_target.plot(kind='barh', ax=axes[1, 0], color=['#ff6b6b', '#4ecdc4'])
axes[1, 0].set_title('Job')
axes[1, 0].legend(['no', 'yes'])

# Education
edu_target = pd.crosstab(df['education'], df['y'], normalize='index') * 100
edu_target.plot(kind='barh', ax=axes[1, 1], color=['#ff6b6b', '#4ecdc4'])
axes[1, 1].set_title('Education')
axes[1, 1].legend(['no', 'yes'])

# Poutcome
pout_target = pd.crosstab(df['poutcome'], df['y'], normalize='index') * 100
pout_target.plot(kind='bar', ax=axes[1, 2], color=['#ff6b6b', '#4ecdc4'])
axes[1, 2].set_title('Résultat précédent')
axes[1, 2].legend(['no', 'yes'])

plt.tight_layout()
plt.show()

**Observations :**
- Classes très déséquilibrées (~88% no et ~11% yes)
- Certaines variables ont des "unknown"
- Corrélations fortes entre variables économiques, donc possible redondance
- La variable "duration" ne sert pas a grand chose car elle est connue après l'appel
- Le résultat de la campagne précédente est important

## 4. Preprocessing

In [ ]:
df_processed = df.copy()

# on retire duration 
df_processed = df_processed.drop('duration', axis=1)
print("Variable duration supprimée")

df_processed['y'] = df_processed['y'].map({'no': 0, 'yes': 1})
print("Cible encodée (no=0, yes=1)")

# Identifier les types de variables
categorical_features = df_processed.select_dtypes(include=['object']).columns.tolist()
numerical_features = df_processed.select_dtypes(include=[np.number]).columns.tolist()
numerical_features.remove('y')

print(f"\nCatégorielles: {categorical_features}")
print(f"Numériques: {numerical_features}")
print(f"\nShape final: {df_processed.shape}")

In [ ]:
# Encodage variables catégorielles
label_encoders = {}

for col in categorical_features:
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col])
    label_encoders[col] = le
    print(f"{col}: {len(le.classes_)} catégories")

df_processed.head()

In [ ]:
# Separation du dataset pour les étapes
X = df_processed.drop('y', axis=1)
y = df_processed['y']

print(f"X: {X.shape}, y: {y.shape}")
print(f"Classe 0: {(y==0).sum()} ({(y==0).sum()/len(y)*100:.1f}%)")
print(f"Classe 1: {(y==1).sum()} ({(y==1).sum()/len(y)*100:.1f}%)")

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nTrain: {X_train.shape}, Test: {X_test.shape}")

In [ ]:
# On normalise les features numériques
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test_scaled[numerical_features] = scaler.transform(X_test[numerical_features])

print(f"Normalisation appliquée sur {len(numerical_features)} features")

## 5. Modélisation

On va tester 5 algorithmes et comparer leurs performances.
1. **Régression Logistique** 
2. **Decision Tree (Arbre de Décision)** 
3. **Naive Bayes** 
4. **Perceptron**
5. **K-Nearest Neighbors (KNN)**

In [ ]:
# Initialisation modèles
models = {
    'Logistic Regression': LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(class_weight='balanced', max_depth=10, random_state=42),
    'Naive Bayes': GaussianNB(),
    'Perceptron': Perceptron(class_weight='balanced', max_iter=1000, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}


In [ ]:
# Entrainement des modeles
import time

trained_models = {}
predictions = {}
prediction_probas = {}
training_times = {}

for name, model in models.items():
    print(f"\n{name}...")
    start = time.time()
    model.fit(X_train_scaled, y_train)
    train_time = time.time() - start
    
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
    
    trained_models[name] = model
    predictions[name] = y_pred
    prediction_probas[name] = y_pred_proba
    training_times[name] = train_time
    
    print(f"Temps: {train_time:.2f}s")
    print(f"Accuracy train: {model.score(X_train_scaled, y_train):.3f}")
    print(f"Accuracy test: {accuracy_score(y_test, y_pred):.3f}")

print("\nEntraînement terminé")

## 6. Évaluation et comparaison

In [ ]:
# Calcul des métriques
results = []

for name in models.keys():
    y_pred = predictions[name]
    y_pred_proba = prediction_probas[name]
    
    results.append({
        'Modèle': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else None,
        'Temps (s)': training_times[name]
    })

results_df = pd.DataFrame(results).sort_values('F1-Score', ascending=False)
results_df

In [ ]:
# visualisation des résultats
fig, axes = plt.subplots(2, 3, figsize=(16, 9))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC', 'Temps (s)']

for idx, metric in enumerate(metrics):
    ax = axes[idx // 3, idx % 3]
    
    if metric == 'ROC-AUC':
        data = results_df[results_df[metric].notna()].sort_values(metric, ascending=False)
    else:
        data = results_df.sort_values(metric, ascending=False if metric != 'Temps (s)' else True)
    
    ax.barh(data['Modèle'], data[metric], alpha=0.8)
    ax.set_title(metric)
    ax.grid(axis='x', alpha=0.3)
    
plt.tight_layout()
plt.show()

In [ ]:
# Matrices de confusion TP FP TN FN
fig, axes = plt.subplots(2, 3, figsize=(15, 9))
axes = axes.ravel()

for idx, name in enumerate(models.keys()):
    cm = confusion_matrix(y_test, predictions[name])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx])
    axes[idx].set_title(name)
    axes[idx].set_xlabel('Prédiction')
    axes[idx].set_ylabel('Réel')

fig.delaxes(axes[5])
plt.tight_layout()
plt.show()

In [ ]:
# Courbes ROC
plt.figure(figsize=(10, 7))

for name in models.keys():
    if prediction_probas[name] is not None:
        fpr, tpr, _ = roc_curve(y_test, prediction_probas[name])
        plt.plot(fpr, tpr, linewidth=2, label=f'{name} (AUC={auc(fpr, tpr):.3f})')

plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Courbes ROC')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## 7. Validation Croisée

In [ ]:
# Cross-validation avec 5-fold
cv_results = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=skf, scoring='f1', n_jobs=-1)
    
    cv_results.append({
        'Modèle': name,
        'F1 Moyen': cv_scores.mean(),
        'Std': cv_scores.std(),
        'Min': cv_scores.min(),
        'Max': cv_scores.max(),
        'Scores': cv_scores
    })
    
    print(f"{name}: {cv_scores.mean():.3f} (+-{cv_scores.std():.3f})")

cv_df = pd.DataFrame(cv_results).drop('Scores', axis=1).sort_values('F1 Moyen', ascending=False)
print(f"\n{cv_df.to_string(index=False)}")

In [ ]:
# visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Barres avec écart type
data = cv_df.sort_values('F1 Moyen', ascending=True)
axes[0].barh(data['Modèle'], data['F1 Moyen'], xerr=data['Std'], capsize=5)
axes[0].set_xlabel('F1-Score Moyen')
axes[0].set_title('CV Results')
axes[0].grid(axis='x', alpha=0.3)

# Boxplot
scores_data = [cv_results[i]['Scores'] for i in range(len(cv_results))]
names = [cv_results[i]['Modèle'] for i in range(len(cv_results))]

axes[1].boxplot(scores_data, labels=names, patch_artist=True)
axes[1].set_ylabel('F1-Score')
axes[1].set_title('Distribution par Fold')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Importance des features avec Decision Tree
dt_model = trained_models['Decision Tree']
importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': dt_model.feature_importances_
}).sort_values('Importance', ascending=False).head(15)

plt.figure(figsize=(10, 6))
plt.barh(range(len(importance)), importance['Importance'].values)
plt.yticks(range(len(importance)), importance['Feature'].values)
plt.xlabel('Importance')
plt.title('Top 15 Features - Decision Tree')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("Features les plus importantes :", importance['Feature'].head(5).tolist())

## 8. Conclusions

### Résultats

D'après ces analyses, le **Decision Tree** donne les meilleurs résultats avec un bon équilibre Precision/Recall et une bonne interprétabilité.

**Comparaison :**
- **Decision Tree** : Meilleures performence, facile à interpréter
- **Régression Logistique** : Bon modèle et assez rapide
- **KNN** : Correct mais lent
- **Naive Bayes** : Rapide mais hypothèse d'indépendance limitante
- **Perceptron** : Performances plus faibles

### Points importants

**Variables influentes :**
- Indicateurs économiques **(euribor3m, nr.employed)**
- Historique contacts **(pdays, poutcome)**

**Problèmes rencontrés :**
- Classes déséquilibrées (88% no / 11% yes)
- Valeurs "unknown" dans certaines variables
- Variable "duration" exclue 

### Améliorations possibles

Optimisation des hyperparamètres :
   - Decision Tree : tester différentes profondeurs
   - KNN : tester différentes valeurs de k 
   - Perceptron : ajuster le learning rate et le nombre d'itérations

Analyse temporelle : 
   - Modèles prenant en compte l'évolution dans le temps  

Feature engineering avancé  

Tester d'autres modèles plus performant 

## Conclusion Finale

Ce projet a démontré l'efficacité de plusieurs algorithmes de machine learning pour prédire le succès des campagnes de télémarketing bancaire. Le **Decision Tree** se distingue comme le meilleur choix, offrant un excellent compromis entre performances et stabilité.

La **Régression Logistique** constitue également une excellente alternative, avec l'avantage d'être très rapide et robuste, bien que légèrement moins performante.

Les modèles développés peuvent significativement améliorer l'efficacité des campagnes marketing en permettant un ciblage plus précis des clients potentiellement intéressés par les dépôts à terme bancaires.

